# Support Vector Machines - Reconhecimento de Imagens

## Reconhecimento de Imagens com Modelos de Regressão Logística e SVM

In [ ]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize
import numpy as np
import pickle
from sklearn.svm import SVC

In [ ]:
# Pré-processamento
def preprocess():
   
    # Carrega o dataset
    mat = loadmat('data/mnist_all.mat')  

    n_feature = mat.get("train1").shape[1]
    n_sample = 0
    for i in range(10):
        n_sample = n_sample + mat.get("train" + str(i)).shape[0]
    n_validation = 1000
    n_train = n_sample - 10 * n_validation

    # Constrói o conjunto de validação
    validation_data = np.zeros((10 * n_validation, n_feature))
    for i in range(10):
        validation_data[i * n_validation:(i + 1) * n_validation, :] = mat.get("train" + str(i))[0:n_validation, :]

    # Labels de validação
    validation_label = np.ones((10 * n_validation, 1))
    for i in range(10):
        validation_label[i * n_validation:(i + 1) * n_validation, :] = i * np.ones((n_validation, 1))

    # Atributos e labels de treino
    train_data = np.zeros((n_train, n_feature))
    train_label = np.zeros((n_train, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("train" + str(i)).shape[0]
        train_data[temp:temp + size_i - n_validation, :] = mat.get("train" + str(i))[n_validation:size_i, :]
        train_label[temp:temp + size_i - n_validation, :] = i * np.ones((size_i - n_validation, 1))
        temp = temp + size_i - n_validation

    # Atributos e labels de teste
    n_test = 0
    for i in range(10):
        n_test = n_test + mat.get("test" + str(i)).shape[0]
    test_data = np.zeros((n_test, n_feature))
    test_label = np.zeros((n_test, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("test" + str(i)).shape[0]
        test_data[temp:temp + size_i, :] = mat.get("test" + str(i))
        test_label[temp:temp + size_i, :] = i * np.ones((size_i, 1))
        temp = temp + size_i

    # Remove atributos que não são relevantes para o modelo
    sigma = np.std(train_data, axis=0)
    index = np.array([])
    for i in range(n_feature):
        if (sigma[i] > 0.001):
            index = np.append(index, [i])
    train_data = train_data[:, index.astype(int)]
    validation_data = validation_data[:, index.astype(int)]
    test_data = test_data[:, index.astype(int)]

    # Normalização dos dados com escala entre 0 e 1
    train_data /= 255.0
    validation_data /= 255.0
    test_data /= 255.0

    return train_data, train_label, validation_data, validation_label, test_data, test_label

In [ ]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
# Computa a função de erro e gradiente de um modelo de 2 classes com Regressão Logística
def blrObjFunction(initialWeights, *args):
    
    train_data, labeli = args

    n_data = train_data.shape[0]
    n_features = train_data.shape[1]

    w = np.reshape(initialWeights,(716, 1))

    dataones = np.ones((n_data,1))
    t_data = np.hstack((dataones,train_data))

    sigin=np.dot(t_data,w)

    thetaN=sigmoid(sigin)

    lntheta=np.log(thetaN)

    product1=np.multiply(labeli,lntheta)

    lntheta2=np.log(1-thetaN)

    product2 = np.multiply((1.0 - labeli),lntheta2)

    s=product1+product2

    e = np.sum(s)

    error = -e/n_data

    ty=thetaN-labeli

    pr1 = np.dot(t_data.T,ty)

    e_grad =pr1/n_data

    error_grad=e_grad.flatten()

    return error, error_grad

In [ ]:
# Previsões
def blrPredict(W, data):
   
    label = np.zeros((data.shape[0], 1))
    dataones = np.ones((data.shape[0],1))
    t_data = np.hstack((dataones,data))

    a = sigmoid(np.dot(t_data, W))

    label = np.argmax(a, axis=1)

    label=label.reshape((data.shape[0],1))

    return label

In [ ]:
# Classificação Multi-classe com Regressão Logística
def mlrObjFunction(params, *args):

    train_data, labeli = args
    n_data = train_data.shape[0]
    n_feature = train_data.shape[1]
    n_class=labeli.shape[1]

    w=params.reshape((n_feature+1,n_class))

    dataones = np.ones((n_data,1))
    t_data = np.hstack((dataones,train_data))

    expin=np.dot(t_data,w)

    e = np.exp(expin / 1.0)
    theta = e / np.sum(e,axis=1).reshape(e.shape[0],1)

    lntheta=np.log(theta)

    er=np.multiply(labeli,lntheta)

    s=np.sum(er)

    error=-s/n_data

    ty=theta-labeli

    egrad=np.dot(t_data.T,ty)

    egrad2=egrad/n_data

    error_grad=egrad2.flatten()

    return error, error_grad

In [ ]:
# Previsões Multi-classe
def mlrPredict(W, data):

    dataones = np.ones((data.shape[0],1))
    t_data = np.hstack((dataones,data))

    a = np.exp(np.dot(t_data, W))

    label = np.argmax(a, axis=1)

    label=label.reshape((data.shape[0],1))

    return label

## Modelo de Regressão Logística

In [ ]:
# Regressão Logística
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

# Número de classes
n_class = 10

# Número de exemplos de treinamento
n_train = train_data.shape[0]

# Número de atributos
n_feature = train_data.shape[1]

Y = np.zeros((n_train, n_class))
for i in range(n_class):
    Y[:, i] = (train_label == i).astype(int).ravel()

# Regressão Logística com Gradiente Descendente
W = np.zeros((n_feature + 1, n_class))
initialWeights = np.zeros((n_feature + 1, 1))
opts = {'maxiter': 100}
for i in range(n_class):
    labeli = Y[:, i].reshape(n_train, 1)
    args = (train_data, labeli)
    nn_params = minimize(blrObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)
    W[:, i] = nn_params.x.reshape((n_feature + 1,))

# Imprime Acurária
predicted_label = blrPredict(W, train_data)
print('\n Acurácia no Conjunto de Dados de Treino: ' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

predicted_label = blrPredict(W, validation_data)
print('\n Acurácia no Conjunto de Dados de Validação: ' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

predicted_label = blrPredict(W, test_data)
print('\n Acurácia no Conjunto de Dados de Teste: ' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')


## Modelo Support Vector Machines 

### Kernel Linear

In [ ]:
# SVM - Kernel Linear
train_label=np.ravel(train_label)
validation_label=np.ravel(validation_label)
test_label=np.ravel(test_label)

print ("***** SVM Kernel Linear *****")

clf = SVC(kernel = 'linear')
clf.fit(train_data, train_label)

# Imprime Acurária
predicted_label= clf.predict(np.array(train_data))
print('\n Acurácia no Conjunto de Dados de Treino usando SVM com Kernel Linear: ' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

predicted_label= clf.predict(np.array(validation_data))
print('\n Acurácia no Conjunto de Dados de Validação usando SVM com Kernel Linear: ' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

predicted_label= clf.predict(np.array(test_data))
print('\n Acurácia no Conjunto de Dados de Teste usando SVM com Kernel Linear: ' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')


### Kernel RBF com Gamma = 1

In [ ]:
# SVM - Kernel RBF com gamma = 1
print ("***** SVM Kernel RBF com gamma = 1 *****")

clf = SVC(kernel='rbf',gamma=1.0)
clf.fit(train_data, train_label)

# Imprime Acurária
predicted_label= clf.predict(np.array(train_data))
print('\n Acurácia no Conjunto de Dados de Treino usando SVM com Kernel RBF com gamma=1: ' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

predicted_label= clf.predict(np.array(validation_data))
print('\n Acurácia no Conjunto de Dados de Validação usando SVM com Kernel RBF com gamma=1: ' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

predicted_label= clf.predict(np.array(test_data))
print('\n Acurácia no Conjunto de Dados de Teste usando SVM com Kernel RBF com gamma=1: ' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')

### Kernel RBF com Gamma default

In [ ]:
# SVM - Kernel RBF com gamma default

print ("***** SVM Kernel RBF com gamma default *****")

clf = SVC(kernel='rbf')
clf.fit(train_data, train_label)

# Imprime Acurária
predicted_label= clf.predict(np.array(train_data))
print('\n Acurácia no Conjunto de Dados de Treino usando SVM com Kernel RBF com gamma default: ' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

predicted_label= clf.predict(np.array(validation_data))
print('\n Acurácia no Conjunto de Dados de Validação usando SVM com Kernel RBF com gamma defaut: ' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

predicted_label= clf.predict(np.array(test_data))
print('\n Acurácia no Conjunto de Dados de Treino usando SVM com Kernel RBF com gamma default: ' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')


### Kernel RBF diferentes valores para C

In [ ]:
# SVM - Kernel RBF com diferentes valores para C

print ("***** SVM Kernel RBF com diferentes valores para C *****")

print ("Valor de C: 1")

clf = SVC(kernel = 'rbf', C = 1.0)
clf.fit(train_data, train_label)

# Imprime Acurária
predicted_label= clf.predict(np.array(train_data))
print('\n Acurácia no Conjunto de Dados de Treino usando SVM com Kernel RBF e C=1: ' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

predicted_label= clf.predict(np.array(validation_data))
print('\n Acurácia no Conjunto de Dados de Validação usando SVM com Kernel RBF e C=1: ' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

predicted_label= clf.predict(np.array(test_data))
print('\n Acurácia no Conjunto de Dados de Teste usando SVM com Kernel RBF e C=1: ' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')

for i in range(10,110,10):

    print ("Valor de C:" + str(i))
    clf = SVC(kernel = 'rbf', C = i)
    clf.fit(train_data, train_label)
    
    predicted_label = clf.predict(np.array(train_data))
    print('\n Acurácia no Conjunto de Dados de Treino usando SVM com Kernel RBF e diferentes valores de C: ' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

    predicted_label = clf.predict(np.array(validation_data))
    print('\n Acurácia no Conjunto de Dados de Validação usando SVM com Kernel RBF e diferentes valores de C: ' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

    predicted_label = clf.predict(np.array(test_data))
    print('\n Acurácia no Conjunto de Dados de Teste usando SVM com Kernel RBF e diferentes valores de C: ' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')